In [1]:
import re
import pandas as pd

from IPython.display import display

In [2]:
df = pd.read_csv("lpdp_qs_the_cwur_complete_num.csv")
df_cs = pd.read_csv("csrankings_cv_cg_viz_2016_2021.csv")
df["Name"] = df["Name"].str.strip()
df_cs["Name"] = df_cs["Name"].str.strip()

In [3]:
# Replace similar name
def replace_similar_name(df_ref: pd.DataFrame, df_proc: pd.DataFrame):
    conflicted_i = set()
    for k, name in df_ref["Name"].items():
        tmp = df_proc["Name"].str.contains(name, regex=False, case=False)
        q = re.sub("[\(\[].*?[\)\]]", "", name).strip()
        tmp |= df_proc["Name"].str.contains(q, regex=False, case=True)
        q = q.replace("The ", "")
        tmp |= df_proc["Name"].str.contains(q, regex=False, case=True)
        q = re.search('\(([^)]+)', name)
        if q is not None:
            q = q.group(1).strip()
            tmp |= df_proc["Name"].eq(q)
        if tmp.sum() > 1:
            matched_i = set(df_proc["Name"][tmp].index.values)
            if matched_i.intersection(conflicted_i):
                continue
            conflicted_i.update(matched_i)
            i = df_proc["Name"][tmp].apply(lambda x: abs(len(x)-len(name))).idxmin()
        else:
            i = tmp.idxmax()
        if tmp.any():
            df_proc.iloc[i, 1] = name
replace_similar_name(df, df_cs)

In [4]:
df_merged = df.merge(df_cs[["Name", "CS Rank", "CV CG Viz Papers Geom Mean Count", "CV CG Viz Faculty"]], how="left", on="Name")
df_merged.to_csv("lpdp_qs_the_cwur_complete_num_csrankings.csv", index=False)
df_merged

,No,Name,Negara,QS 2022,CWUR 2021-2022 Rank,THE Rank 2022,CS Rank,CV CG Viz Papers Geom Mean Count,CV CG Viz Faculty
0,1,University of Oxford,Inggris,2.0,5.0,1.0,193.0,1.8,8.0
1,2,Harvard University,Amerika Serikat,5.0,1.0,2.0,118.0,2.7,4.0
2,3,Stanford University,Amerika Serikat,3.0,3.0,4.0,2.0,12.9,23.0
3,4,Massachusetts Institute of Technology (MIT),Amerika Serikat,1.0,2.0,5.0,19.0,8.4,26.0
4,5,University of Cambridge,Inggris,3.0,4.0,5.0,144.0,2.3,7.0
...,...,...,...,...,...,...,...,...,...
153,154,Uppsala University,Swedia,124.0,88.0,131.0,344.0,1.1,2.0
154,155,National Tsing Hua University,Taiwan,180.0,382.0,351.0,162.0,2.1,6.0
155,156,National Yang Ming Chiao Tung University,Taiwan,268.0,455.0,301.0,NaN,NaN,NaN
156,157,Taipei Medical University,Taiwan,407.0,712.0,201.0,NaN,NaN,NaN
